In [6]:
import numpy as np
import gudhi as gd

simplices_list = [[1, 2, 3]]

# Create the simplex tree
Rips_simplex_tree_sample = gd.SimplexTree()
for simplex in simplices_list:
    Rips_simplex_tree_sample.insert(simplex)

# Finalize the simplex tree
Rips_simplex_tree_sample.initialize_filtration()

rips_list = list(Rips_simplex_tree_sample.get_filtration())

# Extract the 0-dimensional simplices (vertices) from the filtered simplex list
simplices = [i[0] for i in rips_list]

print("Lenth of Filtered Simplical Complex")
print(len(simplices))

Lenth of Filtered Simplical Complex
7


/var/folders/40/zmgqdh5124d3j5h3bdpln8m80000gn/T/ipykernel_47656/2410246647.py:12: DeprecationWarning: Since Gudhi 3.2, calling SimplexTree.initialize_filtration is unnecessary.
  Rips_simplex_tree_sample.initialize_filtration()


In [7]:

# Create a dictionary to map vertices to alpha values so then we dont have to worry about loosing our alpha values
vertex_to_alpha = {tuple(rips[0]): rips[1] for rips in rips_list}

print("Simplex to Alpha:")
for vertex, alpha in vertex_to_alpha.items():
    print(f"Simplex: {vertex}, Alpha: {alpha}")

Simplex to Alpha:
Simplex: (1,), Alpha: 0.0
Simplex: (2,), Alpha: 0.0
Simplex: (1, 2), Alpha: 0.0
Simplex: (3,), Alpha: 0.0
Simplex: (1, 3), Alpha: 0.0
Simplex: (2, 3), Alpha: 0.0
Simplex: (1, 2, 3), Alpha: 0.0


In [8]:
# Extract the edges [a, b] and triangles [c, d, e] from the simplicial complex
edges = []
triangles = []
vertices=[]

for simplex in rips_list:
    vertices = simplex[0]
    
    if len(vertices) == 2:
        edges.append(vertices)
    elif len(vertices) == 3:
        triangles.append(vertices)
    else:
        vertices.append(vertices)

        
#Create a boundary Matrix        
ne = len(edges)
nt = len(triangles)
nv = len(vertices)

num_rows= ne+nt+nv
num_cols=ne+nt+nv

boundary_matrix=np.zeros((num_rows, num_cols), dtype=int)


#Split it up and just deal with the edges and verticies first because the triangles are going to zero out, same will happen with the edges and edges and verticies and verticies

for i,edge in enumerate(edges):
    a,b=edge #a and b are the two verticies that make up the edge
    for j,vertex in enumerate(vertices):
        c=vertex # c d and e are the verticies that make up the triangle
        if b == c:
            boundary_matrix[j,nv+i]=1
        if a == c:
            boundary_matrix[j,nv+i]=-1
        
        
#now deal with the edges and verticies
for i, triangle in enumerate(triangles):
    e,f,g=triangle #e,f,g are verticies in the triangle
    for j,edge in enumerate(edges):
        h,k=edge
        if (h,k) == (e,f):
            boundary_matrix[nv+j,(ne+nv)+i]=1
        if (h,k) == (f,g):
            boundary_matrix[nv+j,(ne+nv)+i]=1
        if (h,k) == (e,g):
            boundary_matrix[nv+j,(ne+nv)+i]=-1


print("Boundary Matrix:")
print(boundary_matrix)
print(len(boundary_matrix))
print(len(boundary_matrix[0]))          
        
for i in range(len(boundary_matrix)):
    print(boundary_matrix[i])




Boundary Matrix:
[[ 0  0  0 -1 -1  0  0]
 [ 0  0  0  1  0 -1  0]
 [ 0  0  0  0  1  1  0]
 [ 0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0]]
7
7
[ 0  0  0 -1 -1  0  0]
[ 0  0  0  1  0 -1  0]
[0 0 0 0 1 1 0]
[0 0 0 0 0 0 1]
[ 0  0  0  0  0  0 -1]
[0 0 0 0 0 0 1]
[0 0 0 0 0 0 0]


In [22]:
#we are now calculating the restricted coboundary matrix

restricted_matrix = boundary_matrix[nv:nv + ne, nv + ne:len(boundary_matrix)]
coboundary_matrix = np.flip(restricted_matrix).T

print("Coboundary Matrix:")
print(coboundary_matrix)
print(len(coboundary_matrix))



Coboundary Matrix:
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
48


In [9]:
#lets play with the torus

T_simplices_list =[[0, 1, 3], [0, 1, 7], [0, 2, 4], [0, 2, 6], [0, 3, 6], [0, 4, 7], [1, 2, 5], [1, 2, 8], [1, 3, 5], [1, 7, 8], [2, 4, 8], [2, 5, 6], [3, 4, 5], [3, 4, 8], [3, 6, 8], [4, 5, 7], [5, 6, 7], [6, 7, 8]]

# Create the simplex tree
T_Rips_simplex_tree_sample = gd.SimplexTree()
for simplex in T_simplices_list:
    T_Rips_simplex_tree_sample.insert(simplex)

# Finalize the simplex tree
Rips_simplex_tree_sample.initialize_filtration()

T_rips_list = list(T_Rips_simplex_tree_sample.get_filtration())

# Extract the 0-dimensional simplices (vertices) from the filtered simplex list
T_simplices = [i[0] for i in rips_list]




/var/folders/40/zmgqdh5124d3j5h3bdpln8m80000gn/T/ipykernel_47656/3386096831.py:11: DeprecationWarning: Since Gudhi 3.2, calling SimplexTree.initialize_filtration is unnecessary.
  Rips_simplex_tree_sample.initialize_filtration()


In [17]:

#more torus time


T_edges = []
T_triangles = []
T_vertices=[]


T_simplices_list =[[0, 1, 3], [0, 1, 7], [0, 2, 4], [0, 2, 6], [0, 3, 6], [0, 4, 7], [1, 2, 5], [1, 2, 8], [1, 3, 5], [1, 7, 8], [2, 4, 8], [2, 5, 6], [3, 4, 5], [3, 4, 8], [3, 6, 8], [4, 5, 7], [5, 6, 7], [6, 7, 8]]


for simplex in T_rips_list:
    T_vert = simplex[0]
    
    if len(T_vert) == 2:
        T_edges.append(T_vert)
    elif len(T_vert) == 3:
        T_triangles.append(T_vert)
    else:
        T_vertices.append(T_vert)

        
#Create a boundary Matrix        
T_ne = len(T_edges)
T_nt = len(T_triangles)
T_nv = len(T_vertices)

num_rows= T_ne+T_nt+T_nv
num_cols= T_ne+T_nt+T_nv

T_boundary_matrix=np.zeros((num_rows, num_cols), dtype=int)


#Split it up and just deal with the edges and verticies first because the triangles are going to zero out, same will happen with the edges and edges and verticies and verticies

for i,edge in enumerate(T_edges):
    a,b=edge #a and b are the two verticies that make up the edge
    for j,vertex in enumerate(T_vertices):
        c=vertex # c d and e are the verticies that make up the triangle
        if b == c:
            T_boundary_matrix[j,T_nv+i]=1
        if a == c:
            T_boundary_matrix[j,T_nv+i]=-1
        
        
#now deal with the edges and verticies
for i, triangle in enumerate(T_triangles):
    e,f,g=triangle #e,f,g are verticies in the triangle
    for j,edge in enumerate(T_edges):
        h,k=edge
        if (h,k) == (e,f):
            T_boundary_matrix[T_nv+j,(T_ne+T_nv)+i]=1
        if (h,k) == (f,g):
            T_boundary_matrix[T_nv+j,(T_ne+T_nv)+i]=1
        if (h,k) == (e,g):
            T_boundary_matrix[T_nv+j,(T_ne+T_nv)+i]=-1


print("Boundary Matrix:")
print(T_boundary_matrix)
print(len(T_boundary_matrix))
print(len(T_boundary_matrix[0]))          
        
for i in range(len(T_boundary_matrix)):
    print(T_boundary_matrix[i])




Boundary Matrix:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
54
54
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 

In [23]:
#we are finding the restricted coboundary matrix for the torus

T_restricted_matrix = T_boundary_matrix[T_nv:T_nv + T_ne, T_nv + T_ne:len(T_boundary_matrix)]
T_coboundary_matrix = np.flip(T_restricted_matrix).T

print("Coboundary Matrix:")
print(T_coboundary_matrix)
print(len(T_coboundary_matrix))

Coboundary Matrix:
[[ 1 -1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 1  0  0  0  0 -1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  1  0 -1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
   0  0  0]
 [ 0  0  1  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
   0  0  0]
 [ 0  0  0  0  1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   1  0  0]
 [ 0  0  0  0  0  0  1 -1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  1 -1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0 -1  0  0  0  0  0  0  0  0  0  0  1  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0 -1  0  0  0  1  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 